In [1]:
%cd ..

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib nbagg
import os.path as op
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau
from keras.models import load_model
from keras.optimizers import Adam
from keras_tqdm import TQDMNotebookCallback
import tensorflow as tf
from tqdm import tqdm_notebook

from data import MaskedUntouched2DSequence
from pdnet import PDNet, InvShiftCropNet
from utils import keras_psnr, keras_ssim

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# paths
train_path = '/media/Zaccharie/UHRes/singlecoil_train/singlecoil_train/'
val_path = '/media/Zaccharie/UHRes/singlecoil_val/'
test_path = '/media/Zaccharie/UHRes/singlecoil_test/'

In [4]:
n_samples_train = 34742
n_samples_val = 7135

n_volumes_train = 973
n_volumes_val = 199

In [5]:
# generators
AF = 4
# MaskedUntouched2DSequence
train_gen = MaskedUntouched2DSequence(train_path, af=AF)
val_gen = MaskedUntouched2DSequence(val_path, af=AF)

In [6]:
run_params = {
#     'n_primal': 5,
#     'n_dual': 5,
#     'n_iter': 10,
#     'n_filters': 32,
    'n_primal': 2,
    'n_dual': 2,
    'n_iter': 2,
    'n_filters': 8,
}

n_epochs = 2
run_id = f'pdnet_subclassed_af{AF}'
chkpt_path = f'checkpoints/{run_id}' + '-{epoch:02d}.hdf5'

In [7]:
chkpt_cback = ModelCheckpoint(chkpt_path, period=50, save_weights_only=True)
log_dir = op.join('logs', run_id)
tboard_cback = TensorBoard(
    log_dir=log_dir, 
    histogram_freq=0, 
    write_graph=True, 
    write_images=False, 
)
lr_on_plat_cback = ReduceLROnPlateau(monitor='val_loss', min_lr=5*1e-5, mode='auto', patience=3)
# fix from https://github.com/bstriner/keras-tqdm/issues/31#issuecomment-516325065
tqdm_cb = TQDMNotebookCallback(metric_format="{name}: {value:e}")
# tqdm_cb.on_train_batch_begin = tqdm_cb.on_batch_begin
# tqdm_cb.on_train_batch_end = tqdm_cb.on_batch_end
# tqdm_cb.on_test_begin = lambda x,y:None
# tqdm_cb.on_test_end = lambda x,y:None

In [8]:
# model = InvShiftCropNet(**run_params)
model = PDNet(**run_params)
# model.build((640, None, 1))
model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=1e-3),
    loss='mean_absolute_error',
    metrics=['mean_squared_error', keras_psnr, keras_ssim],
)

In [9]:
# model.fit_generator(
#     train_gen, 
#     steps_per_epoch=n_volumes_train, 
#     epochs=n_epochs,
#     validation_data=val_gen,
#     validation_steps=n_volumes_val,
#     verbose=0,
#     callbacks=[tqdm_cb, tboard_cback, chkpt_cback, lr_on_plat_cback],
#     max_queue_size=100,
#     use_multiprocessing=True,
#     workers=35,
# )

In [ ]:
# overfitting trials

data = train_gen[0]
val_data = val_gen[0]
model.fit(
    x=data[0], 
    y=data[1], 
#     validation_data=val_data, 
    batch_size=data[0][0].shape[0], 
    epochs=1000, 
    verbose=2, 
    shuffle=False,
)

Epoch 1/1000
36/36 - 2s - loss: 8.1266e-06 - mean_squared_error: 1.2846e-10 - keras_psnr: 37.9580 - keras_ssim: 0.8831
Epoch 2/1000
36/36 - 2s - loss: 8.0990e-06 - mean_squared_error: 1.2747e-10 - keras_psnr: 37.9865 - keras_ssim: 0.8835
Epoch 3/1000
36/36 - 2s - loss: 8.0718e-06 - mean_squared_error: 1.2648e-10 - keras_psnr: 38.0148 - keras_ssim: 0.8840
Epoch 4/1000
36/36 - 2s - loss: 8.0448e-06 - mean_squared_error: 1.2551e-10 - keras_psnr: 38.0429 - keras_ssim: 0.8844
Epoch 5/1000
36/36 - 2s - loss: 8.0181e-06 - mean_squared_error: 1.2456e-10 - keras_psnr: 38.0709 - keras_ssim: 0.8848
Epoch 6/1000
36/36 - 2s - loss: 7.9918e-06 - mean_squared_error: 1.2362e-10 - keras_psnr: 38.0986 - keras_ssim: 0.8852
Epoch 7/1000
36/36 - 2s - loss: 7.9659e-06 - mean_squared_error: 1.2270e-10 - keras_psnr: 38.1261 - keras_ssim: 0.8855
Epoch 8/1000
36/36 - 2s - loss: 7.9404e-06 - mean_squared_error: 1.2179e-10 - keras_psnr: 38.1533 - keras_ssim: 0.8859
Epoch 9/1000
36/36 - 2s - loss: 7.9152e-06 - mea

Epoch 70/1000
36/36 - 2s - loss: 7.0738e-06 - mean_squared_error: 9.2854e-11 - keras_psnr: 39.1861 - keras_ssim: 0.8999
Epoch 71/1000
36/36 - 2s - loss: 7.0668e-06 - mean_squared_error: 9.2635e-11 - keras_psnr: 39.1953 - keras_ssim: 0.9000
Epoch 72/1000
36/36 - 2s - loss: 7.0599e-06 - mean_squared_error: 9.2419e-11 - keras_psnr: 39.2043 - keras_ssim: 0.9001
Epoch 73/1000
36/36 - 2s - loss: 7.0531e-06 - mean_squared_error: 9.2206e-11 - keras_psnr: 39.2133 - keras_ssim: 0.9002
Epoch 74/1000
36/36 - 2s - loss: 7.0464e-06 - mean_squared_error: 9.1997e-11 - keras_psnr: 39.2221 - keras_ssim: 0.9003
Epoch 75/1000
36/36 - 2s - loss: 7.0399e-06 - mean_squared_error: 9.1792e-11 - keras_psnr: 39.2309 - keras_ssim: 0.9005
Epoch 76/1000
36/36 - 2s - loss: 7.0334e-06 - mean_squared_error: 9.1590e-11 - keras_psnr: 39.2395 - keras_ssim: 0.9006
Epoch 77/1000
36/36 - 2s - loss: 7.0270e-06 - mean_squared_error: 9.1391e-11 - keras_psnr: 39.2479 - keras_ssim: 0.9007
Epoch 78/1000
36/36 - 2s - loss: 7.0208e